In [1]:
import os
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from langchain_openai import ChatOpenAI
import yfinance as yf
from dataclasses import dataclass
import logging

# ---------------------------
# 로깅 설정
# ---------------------------
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# 환경 변수 로드 및 검증
# ---------------------------
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key not found in environment variables")


# ---------------------------
# 분석에 필요한 설정 데이터 클래스
# ---------------------------
@dataclass
class AnalysisConfig:
    """기술적 분석과 재무 분석 시 필요한 기간, 파라미터 등의 설정을 저장하는 데이터 클래스."""

    period_years: int = 5
    interval: str = (
        "1wk"  # yfinance 호출 시 주간 간격(1wk), 일간(1d), 월간(1mo) 등 사용 가능
    )
    moving_averages: List[int] = None
    rsi_period: int = 14
    volume_ma_period: int = 20

    def __post_init__(self):
        if self.moving_averages is None:
            self.moving_averages = [50, 200]  # 기본 이동평균 기간


# ---------------------------
# 작업(Tasks)을 정의하는 클래스
# ---------------------------
class Tasks:
    # 재무 분석 작업
    def financial_analysis(self, agent):
        return Task(
            description="""재무 분석을 수행합니다. 5년간의 핵심 재무지표(매출, 이익, 현금흐름 등)를 살펴보고
            업계 대비 경쟁 우위나 리스크 등을 파악합니다.""",
            agent=agent,
            expected_output="""주요 재무지표와 성장성, 수익성, 재무안정성 등을 점검한 뒤,
            기업의 재무 건전성 및 투자 리스크를 정리해 반환합니다.""",
        )

    # 기술적 분석 작업
    def technical_analysis(self, agent):
        return Task(
            description="""과거 5년 주가 데이터를 기반으로 RSI, MACD 등 지표를 계산하고,
            지지선/저항선을 포함한 주요 가격 레벨을 찾습니다.""",
            agent=agent,
            expected_output="""기술적 분석 지표(RSI, MACD, 이동평균)와 지지·저항선, 거래량 추세를 종합 분석하여
            향후 주가 방향성과 변동성에 대한 견해를 반환합니다.""",
        )

    # 최종 투자 판단(STRONG BUY / BUY / HOLD / SELL / STRONG SELL) 작업
    def investment_recommendation(self, agent, context_tasks):
        return Task(
            description="""모든 분석 결과를 종합해 최종 투자 의사결정을 내립니다.
            (STRONG BUY / BUY / HOLD / SELL / STRONG SELL)와 그 근거, 목표주가, 리스크, 시점 등 명시.""",
            agent=agent,
            context_tasks=context_tasks,
            expected_output="""최종적으로 BUY / HOLD / SELL 등 투자의견과 구체적 가격 목표, 주된 리스크 요인을
            요약하여 결과를 반환합니다.""",
        )


# ---------------------------
# 마크다운 파일에 결과 저장 함수
# ---------------------------
def save_analysis_to_markdown(result: str, ticker: str) -> Path:
    """
    분석 결과를 마크다운 파일로 저장합니다.
    :param result: 분석 결과 문자열
    :param ticker: 종목 티커
    :return: 저장된 파일 경로
    """
    reports_dir = Path("reports")
    reports_dir.mkdir(exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = reports_dir / f"{ticker}_analysis_{timestamp}.md"

    content = f"""# Investment Analysis Report for {ticker}
Generated on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

## Analysis Summary
{result}

---
*This report was generated automatically by the Stock Analysis System*
"""

    with open(filename, "w", encoding="utf-8") as f:
        f.write(content)

    return filename


# ---------------------------
# 재무 지표 계산 클래스
# ---------------------------
class FinancialMetrics:
    """기업의 재무 상태를 파악하기 위한 다양한 재무 지표 계산 로직을 담은 클래스"""

    @staticmethod
    def calculate_ratios(
        balance_sheet: pd.DataFrame, income_stmt: pd.DataFrame
    ) -> Dict:
        """
        예시로 현재비율, 부채비율, ROE, Quick Ratio 등을 계산합니다.
        :param balance_sheet: yfinance에서 가져온 대차대조표 데이터
        :param income_stmt: yfinance에서 가져온 손익계산서 데이터
        :return: 계산된 지표 딕셔너리
        """
        try:
            # balance_sheet와 income_stmt가 비거나, 특정 행이 없는 경우 예외 처리
            if balance_sheet is None or balance_sheet.empty:
                raise ValueError("Balance Sheet 데이터가 존재하지 않습니다.")
            if income_stmt is None or income_stmt.empty:
                raise ValueError("Income Statement 데이터가 존재하지 않습니다.")

            # 일반적으로 yfinance의 재무제표는 컬럼이 기간, 인덱스가 항목으로 구성
            # 가장 최근 컬럼(보통 0번째)이 존재하는지 확인
            recent_col = balance_sheet.columns[0]
            # 필요한 계정과목이 모두 존재하는지 검사
            required_items = [
                "Total Current Assets",
                "Total Current Liabilities",
                "Total Liab",
                "Total Stockholder Equity",
            ]
            for item in required_items:
                if item not in balance_sheet.index:
                    raise KeyError(f"{item} 항목이 존재하지 않습니다.")

            # 계산 예시
            current_assets = balance_sheet.loc["Total Current Assets", recent_col]
            current_liab = balance_sheet.loc["Total Current Liabilities", recent_col]
            total_liab = balance_sheet.loc["Total Liab", recent_col]
            total_equity = balance_sheet.loc["Total Stockholder Equity", recent_col]

            # Income stmt에서 Net Income 가져오기 (예외 처리)
            if "Net Income" not in income_stmt.index:
                raise KeyError("Net Income 항목이 Income Statement에 없습니다.")
            net_income = income_stmt.loc["Net Income", recent_col]

            # 재무비율 계산
            current_ratio = current_assets / current_liab if current_liab != 0 else None
            debt_to_equity = total_liab / total_equity if total_equity != 0 else None
            quick_ratio = (
                (current_assets * 0.8) / current_liab if current_liab != 0 else None
            )  # 단순 예시
            roe = net_income / total_equity if total_equity != 0 else None

            return {
                "current_ratio": current_ratio,
                "debt_to_equity": debt_to_equity,
                "quick_ratio": quick_ratio,
                "roe": roe,
            }

        except Exception as e:
            logger.error(f"Error calculating financial ratios: {e}")
            return {}


# ---------------------------
# 기술적 분석 관련 클래스
# ---------------------------
class TechnicalAnalysis:
    """RSI, MACD 등 기술적 지표를 계산하고 지지/저항선을 찾는 기능을 제공합니다."""

    @staticmethod
    def calculate_rsi(data: pd.DataFrame, period: int = 14) -> pd.Series:
        """
        Wilder's Smoothing 방식을 간단히 구현한 RSI 계산
        :param data: 주가 히스토리 DataFrame
        :param period: RSI 계산에 사용할 기간
        """
        # 종가 차이 계산
        delta = data["Close"].diff()
        # 상승분/하락분
        gain = delta.clip(lower=0)
        loss = -1 * delta.clip(upper=0)

        # 지수 이동평균(EMA) 적용
        alpha = 1 / period
        avg_gain = gain.ewm(alpha=alpha, adjust=False).mean()
        avg_loss = loss.ewm(alpha=alpha, adjust=False).mean()

        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    @staticmethod
    def calculate_macd(
        data: pd.DataFrame,
        short_window: int = 12,
        long_window: int = 26,
        signal_window: int = 9,
    ):
        """
        MACD 지표 계산: 단기 EMA - 장기 EMA, 그리고 시그널 라인(9일 EMA)
        :param data: 주가 히스토리 DataFrame
        :param short_window: 단기 EMA 기간
        :param long_window: 장기 EMA 기간
        :param signal_window: 시그널 EMA 기간
        """
        close = data["Close"]
        ema_short = close.ewm(span=short_window, adjust=False).mean()
        ema_long = close.ewm(span=long_window, adjust=False).mean()
        macd_line = ema_short - ema_long
        signal_line = macd_line.ewm(span=signal_window, adjust=False).mean()
        macd_hist = macd_line - signal_line

        return macd_line, signal_line, macd_hist

    @staticmethod
    def calculate_support_resistance(
        data: pd.DataFrame, window: int = 20
    ) -> Tuple[float, float]:
        """
        지정된 window 기간 내 최저가/최고가를 기반으로 지지선/저항선을 추정
        :param data: 주가 히스토리 DataFrame
        :param window: 지지선/저항선 산출을 위한 기간
        :return: (support, resistance) 튜플
        """
        # 결측치 방지를 위해 최소 window 길이 이상의 데이터가 있는지 체크
        if len(data) < window:
            return data["Low"].min(), data["High"].max()

        rolling_min = data["Low"].rolling(window=window).min()
        rolling_max = data["High"].rolling(window=window).max()
        return rolling_min.iloc[-1], rolling_max.iloc[-1]


# ---------------------------
# 종합 주가 분석 툴
# ---------------------------
class ComprehensivePriceTool(BaseTool):
    name: str = "Comprehensive Price Analysis"
    description: str = (
        "과거 주가와 기술적 지표(RSI, MACD 등)를 종합적으로 산출하여 반환합니다."
    )

    def __init__(self, config: AnalysisConfig):
        super().__init__()
        self._config = config

    def _run(self, ticker: str) -> str:
        """
        yfinance를 통해 ticker의 과거 5년 주가 데이터를 가져온 뒤,
        이동평균, RSI, MACD, 지지/저항선 등을 계산한 결과를 문자열로 반환
        """
        try:
            stock = yf.Ticker(ticker)
            history = stock.history(
                period=f"{self._config.period_years}y", interval=self._config.interval
            )

            if history.empty:
                return "No price data available for this ticker"

            # 이동평균 계산
            for ma in self._config.moving_averages:
                history[f"MA_{ma}"] = history["Close"].rolling(window=ma).mean()

            # RSI 계산
            history["RSI"] = TechnicalAnalysis.calculate_rsi(
                history, self._config.rsi_period
            )

            # MACD 계산
            macd_line, signal_line, macd_hist = TechnicalAnalysis.calculate_macd(
                history
            )
            history["MACD"] = macd_line
            history["MACD_signal"] = signal_line
            history["MACD_hist"] = macd_hist

            # 거래량 이동평균
            history["Volume_MA"] = (
                history["Volume"].rolling(window=self._config.volume_ma_period).mean()
            )

            # 지지/저항선
            support, resistance = TechnicalAnalysis.calculate_support_resistance(
                history
            )

            # 간단한 분석 요약 생성
            analysis_summary = {
                "current_price": float(history["Close"].iloc[-1]),
                "support_level": float(support),
                "resistance_level": float(resistance),
                "RSI": float(history["RSI"].iloc[-1]),
                "MACD": float(history["MACD"].iloc[-1]),
                "MACD_signal": float(history["MACD_signal"].iloc[-1]),
                "Volume": int(history["Volume"].iloc[-1]),
                "Volume_MA": float(history["Volume_MA"].iloc[-1]),
            }

            return str(analysis_summary)
        except Exception as e:
            logger.error(f"Error in price analysis: {e}")
            return f"Error analyzing price data: {str(e)}"


# ---------------------------
# 재무 분석 툴
# ---------------------------
class FinancialMetricsTool(BaseTool):
    name: str = "Financial Metrics Analysis"
    description: str = (
        "종합 재무제표(대차대조표, 손익계산서, 현금흐름표)를 분석하고 핵심 비율을 산출합니다."
    )

    def _run(self, ticker: str) -> str:
        """
        yfinance에서 제공하는 balance_sheet, income_stmt, cashflow 등을 활용하여
        핵심 재무지표(유동성, 부채비율, ROE 등)와 성장률, 마진 등을 추정 후 반환
        """
        try:
            stock = yf.Ticker(ticker)

            balance_sheet = stock.balance_sheet
            income_stmt = stock.financials  # 종종 .income_stmt 대신 .financials를 사용
            cash_flow = stock.cashflow

            # 데이터가 없는 경우 예외 처리
            if balance_sheet is None or balance_sheet.empty:
                return "No balance sheet data available."
            if income_stmt is None or income_stmt.empty:
                return "No income statement data available."
            if cash_flow is None or cash_flow.empty:
                return "No cash flow data available."

            # 핵심 비율 계산
            key_ratios = FinancialMetrics.calculate_ratios(balance_sheet, income_stmt)

            # 매출 성장률(평균)
            if "Total Revenue" in income_stmt.index:
                revenue_series = income_stmt.loc["Total Revenue"]
                revenue_growth = revenue_series.pct_change().mean()
            else:
                revenue_growth = None

            # 순이익률(평균)
            if (
                "Net Income" in income_stmt.index
                and "Total Revenue" in income_stmt.index
            ):
                profit_margin = (
                    income_stmt.loc["Net Income"] / income_stmt.loc["Total Revenue"]
                ).mean()
            else:
                profit_margin = None

            metrics = {
                "key_ratios": key_ratios,
                "revenue_growth_avg": revenue_growth,
                "profit_margin_avg": profit_margin,
            }

            return str(metrics)
        except Exception as e:
            logger.error(f"Error in financial analysis: {e}")
            return f"Error analyzing financial data: {str(e)}"


# ---------------------------
# 고급 에이전트 정의 클래스
# ---------------------------
class AdvancedAgents:
    """
    에이전트를 생성하여 각각의 역할(재무 분석, 기술적 분석, 종합 판단)을 수행하도록 세팅하는 클래스
    """

    def __init__(self, config: AnalysisConfig):
        self.config = config

    def financial_analyst(self) -> Agent:
        return Agent(
            role="Senior Financial Analyst",
            goal="기업 재무 상태를 장기적 관점에서 평가하고, 리스크와 기회를 심층 분석한다.",
            backstory="""20년 경력의 재무분석 전문가로, 기업의 펀더멘털을 정교하게 분석하여 
                        장기적 투자 가치와 리스크를 식별해 낸다.""",
            tools=[FinancialMetricsTool()],
            verbose=True,
        )

    def technical_analyst(self) -> Agent:
        return Agent(
            role="Technical Analysis Expert",
            goal="주가 패턴과 기술적 지표를 다각도로 분석하여 추세와 변곡점을 찾아낸다.",
            backstory="""차트 분석 전문가로, RSI와 MACD 등을 이용해 시장 심리를 꿰뚫고 
                        상승/하락 국면 전환 시점을 예리하게 포착한다.""",
            tools=[ComprehensivePriceTool(self.config)],
            verbose=True,
        )

    def hedge_fund_manager(self) -> Agent:
        return Agent(
            role="Hedge Fund Manager",
            goal="재무 및 기술적 분석 결과를 종합해 최적의 투자 의사결정을 내린다.",
            backstory="""글로벌 시장에서 25년 이상 펀드를 운용한 베테랑 매니저. 
                         기업의 펀더멘털과 시장 센티먼트 양쪽을 고루 고려하여 
                         Buy/Hold/Sell 결정을 과감하고 정확하게 내린다.""",
            tools=[],  # 이미 다른 에이전트의 분석 결과를 활용하므로 별도 툴 불필요
            verbose=True,
        )


# ---------------------------
# Crew를 구성하고 실행하는 함수
# ---------------------------
def create_analysis_crew(
    ticker: str, config: AnalysisConfig = AnalysisConfig()
) -> Tuple[str, Path]:
    """
    주어진 티커를 분석하기 위해 재무 분석, 기술적 분석, 최종 투자 의사결정 과정을 순서대로 진행하고,
    그 결과를 마크다운 파일에 저장한 뒤 결과 문자열과 파일 경로를 반환합니다.
    """
    try:
        # 에이전트 및 작업(Tasks) 초기화
        agents = AdvancedAgents(config)
        tasks = Tasks()

        financial_analyst = agents.financial_analyst()
        technical_analyst = agents.technical_analyst()
        hedge_fund_manager = agents.hedge_fund_manager()

        # 작업(테스크) 정의
        financial_task = tasks.financial_analysis(financial_analyst)
        technical_task = tasks.technical_analysis(technical_analyst)
        recommend_task = tasks.investment_recommendation(
            hedge_fund_manager,
            context_tasks=[financial_task, technical_task],
        )

        # Crew 생성 및 설정
        crew = Crew(
            agents=[financial_analyst, technical_analyst, hedge_fund_manager],
            tasks=[financial_task, technical_task, recommend_task],
            verbose=True,
            process=Process.sequential,
            manager_llm=ChatOpenAI(
                model_name="gpt-4o-mini",  # 실제 지원되는 OpenAI 모델명으로 기입
                temperature=0.6,
                api_key=OPENAI_API_KEY,
            ),
        )

        # 작업 실행
        result = crew.kickoff(inputs={"company": ticker})

        # 결과를 마크다운 파일로 저장
        report_file = save_analysis_to_markdown(result, ticker)

        return result, report_file

    except Exception as e:
        logger.error(f"Error in analysis execution: {e}")
        raise


# ---------------------------
# 메인 실행부 (직접 스크립트 실행 시)
# ---------------------------
if __name__ == "__main__":
    try:
        # 분석 파라미터 설정
        config = AnalysisConfig(
            period_years=5,  # 5년치 데이터
            interval="1wk",  # 주별 데이터
            moving_averages=[50, 100, 200],
            rsi_period=14,
            volume_ma_period=20,
        )

        # 실제 분석 실행 예시 - 삼성전자(005930.KS) 티커
        result, report_file = create_analysis_crew("005930.KS", config)

        print(f"Analysis completed. Report saved to: {report_file}")
        print("\n=== Final Analysis Result ===")
        print(result)

    except Exception as e:
        logger.error(f"Analysis failed: {e}")

LLM value is None
LLM value is None
LLM value is None


23:24:09 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai


# Agent: Senior Financial Analyst
## Task: 재무 분석을 수행합니다. 5년간의 핵심 재무지표(매출, 이익, 현금흐름 등)를 살펴보고
            업계 대비 경쟁 우위나 리스크 등을 파악합니다.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
23:24:11 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
ERROR:__main__:Error calculating financial ratios: 'Total Current Assets 항목이 존재하지 않습니다.'
C:\Users\Jonathan\AppData\Local\Temp\ipykernel_1424\1817205661.py:363: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  revenue_growth = revenue_series.pct_change().mean()
C:\Users\Jonathan\AppData\Local\Temp\ipykernel_1424\1817205661.py:363: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('futu



# Agent: Senior Financial Analyst
## Thought: I need to gather the necessary financial metrics for analysis to evaluate the fundamental performance of the company over the last five years.
## Using tool: Financial Metrics Analysis
## Tool Input: 
"{\"ticker\": \"AAPL\"}"
## Tool Output: 
{'key_ratios': {}, 'revenue_growth_avg': np.float64(-0.015827622992996615), 'profit_margin_avg': np.float64(0.2511723102378953)}


...



# Agent: Technical Analysis Expert
## Using tool: Comprehensive Price Analysis
## Tool Input: 
"{\"ticker\": \"NFLX\"}"
## Tool Output: 
{'current_price': 977.5900268554688, 'support_level': 660.7999877929688, 'resistance_level': 999.0, 'RSI': 72.72911309186851, 'MACD': 65.75239399309032, 'MACD_signal': 63.58866777540316, 'Volume': 36419000, 'Volume_MA': 16095800.0}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
23:24:34 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
23:24:34 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai




# Agent: Technical Analysis Expert
## Final Answer: 
1. **Current Price**: The current price of NFLX is approximately $977.59.
  
2. **Support and Resistance Levels**: 
   - Support Level: $660.80
   - Resistance Level: $999.00

3. **Relative Strength Index (RSI)**: The RSI value is 72.73, which indicates that the stock is currently in an overbought condition, suggesting a potential price correction or pullback.

4. **MACD Analysis**: 
   - MACD Value: 65.75
   - MACD Signal: 63.59
   This indicates a bullish momentum, as the MACD is above the signal line. However, with RSI being overbought, it's crucial to watch for signs of a reversal or weakening momentum.

5. **Volume Analysis**: The current trading volume is 36,419,000, which is significantly higher than the 20-day moving average volume of 16,095,800. This suggests strong trading activity and may indicate investor interest, but it can also contribute to volatility.

**Conclusion**: 
Based on the current technical indicators, NFL

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
23:24:42 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler




# Agent: Hedge Fund Manager
## Final Answer: 
Based on the analysis of the current market conditions for Netflix (NFLX), my investment recommendation is **HOLD**. 

### Investment Recommendation: HOLD
- **Current Price**: $977.59
- **Target Price**: $950.00 (short-term pullback target based on overbought condition)
- **Support Level**: $660.80
- **Resistance Level**: $999.00
- **Key Indicators**: 
  - **RSI**: 72.73 (indicates overbought condition)
  - **MACD**: 65.75 (bullish momentum, but requires monitoring)
  - **Volume**: 36,419,000 (significantly above the 20-day average, indicating strong activity)

### Rationale: 
1. **Technical Indicators**: The RSI indicates that the stock is currently overbought; hence, a potential price correction may be imminent. Although there is bullish momentum suggested by the MACD, the overbought condition necessitates caution.
   
2. **Market Sentiment**: Strong trading volume reflects investor interest, but it also implies increased volatility. Th